In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os

BASE_DIR = '/content/drive/MyDrive/AeroSound-FaultNet'
os.chdir(BASE_DIR)
print("Working directory:", os.getcwd())


Mounted at /content/drive
Working directory: /content/drive/MyDrive/AeroSound-FaultNet


In [2]:
!pip install -q tensorflow tensorflow-hub tensorflow-io librosa pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 105.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import os

manifests = 'data/manifests'

train_df = pd.read_csv(os.path.join(manifests, 'train.csv'))
val_df   = pd.read_csv(os.path.join(manifests, 'val.csv'))
test_df  = pd.read_csv(os.path.join(manifests, 'test.csv'))

print(len(train_df), len(val_df), len(test_df))


6557 1412 1402


In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
import pandas as pd

# Load YAMNet
YAMNET_MODEL_HANDLE = "https://tfhub.dev/google/yamnet/1"
yamnet = hub.load(YAMNET_MODEL_HANDLE)

label_map = {'healthy': 0, 'faulty': 1}

def load_wav_16k(path):
    """Load audio as 16k mono float32 numpy array. Raises if totally broken."""
    wav, sr = librosa.load(path, sr=16000, mono=True)
    wav = wav.astype(np.float32)
    if wav.size == 0:
        raise ValueError(f"Empty waveform: {path}")
    return wav

def wav_to_embedding(wav):
    """
    Convert waveform (1D np.array) -> 1024-dim embedding.
    Handles weird shapes defensively so training doesn't crash.
    """
    # YAMNet expects a tf.float32 tensor
    wav_tf = tf.convert_to_tensor(wav, dtype=tf.float32)

    scores, embeddings, spectrogram = yamnet(wav_tf)

    # Convert to numpy
    emb_np = embeddings.numpy()

    # Expected shape: [num_patches, 1024]
    if emb_np.ndim == 2 and emb_np.shape[1] == 1024:
        emb = emb_np.mean(axis=0)
    elif emb_np.ndim == 1 and emb_np.shape[0] == 1024:
        # Rare case: already a single 1024-d vector
        emb = emb_np
    else:
        # Fallback: don't blow up training; log and return zeros
        print(f"[WARN] Unexpected embedding shape {emb_np.shape}, using zeros instead.")
        emb = np.zeros(1024, dtype=np.float32)

    return emb.astype(np.float32)


In [13]:
def make_dataset(csv_path, batch=32, shuffle=True):
    df = pd.read_csv(csv_path)
    paths = df['path'].tolist()
    labels = [label_map[x] for x in df['label']]

    base_ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def _load_py(path, label):
        """
        Runs in Python via tf.py_function – can use librosa, numpy, etc.
        Must be defensive: skip / fix bad files instead of crashing.
        """
        path_str = path.numpy().decode("utf-8")

        try:
            wav = load_wav_16k(path_str)
            emb = wav_to_embedding(wav)
        except Exception as e:
            # If something goes wrong (corrupt file, etc.), fall back to zeros
            print(f"[WARN] Failed to process {path_str}: {e}")
            emb = np.zeros(1024, dtype=np.float32)

        return emb, np.int64(label)

    def wrapper(path, label):
        emb, lab = tf.py_function(
            _load_py, [path, label], [tf.float32, tf.int64]
        )
        emb.set_shape([1024])  # fix static shape for Keras
        lab.set_shape([])
        return emb, lab

    ds = base_ds.map(wrapper, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1024)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset('data/manifests/train.csv', batch=32, shuffle=True)
val_ds   = make_dataset('data/manifests/val.csv',   batch=32, shuffle=False)
test_ds  = make_dataset('data/manifests/test.csv',  batch=32, shuffle=False)


In [14]:
from tensorflow.keras import layers, models

def build_model():
    inputs = layers.Input(shape=(1024,))
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    return models.Model(inputs, outputs)

model = build_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 278,978 (1.06 MB)

 Trainable params: 278,978 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 84s 364ms/step - accuracy: 0.9335 - loss: 0.1940 - val_accuracy: 0.9441 - val_loss: 0.1796
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 76s 322ms/step - accuracy: 0.9503 - loss: 0.1470 - val_accuracy: 0.9426 - val_loss: 0.1686
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 80s 343ms/step - accuracy: 0.9654 - loss: 0.1029 - val_accuracy: 0.9533 - val_loss: 0.1587
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 78s 323ms/step - accuracy: 0.9703 - loss: 0.0871 - val_accuracy: 0.9483 - val_loss: 0.1690
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 80s 319ms/step - accuracy: 0.9716 - loss: 0.0773 - val_accuracy: 0.9518 - val_loss: 0.1737
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 75s 321ms/step - accuracy: 0.9760 - loss: 0.0688 - val_accuracy: 0.9540 - val_loss: 0.1704
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 82s 356ms/step - accuracy: 0.9742 - loss: 0.0680 - val_accuracy: 0.9568 - val_loss: 0.1748
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 75s 318ms/step - accuracy: 0.9750 - loss: 0

In [19]:
import pandas as pd

# Assuming model and test_ds are already defined
pred_proba = model.predict(test_ds)
pred_classes = np.argmax(pred_proba, axis=1)

test_df = pd.read_csv('data/manifests/test.csv')
test_df['pred'] = pred_classes
test_df['pred_label'] = test_df['pred'].map({0:'healthy', 1:'faulty'})

# Set the display option for max rows
pd.set_option('display.max_rows', 10)

# Display the first 10 rows
test_df.head(10)


44/44 ━━━━━━━━━━━━━━━━━━━━ 14s 317ms/step


,path,label,sr,snr,fp,group,pred,pred_label
0,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,healthy,16000,clean,18b19a5700e73f134b21d3ea2c3cd4b7,000000_2022-12-08_10-11-50_0_0,0,healthy
1,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,healthy,16000,30,a76485fac9dc69487f7bdbe749cda040,000000_2022-12-08_10-11-50_0_0,0,healthy
2,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,healthy,16000,20,73f27bf899965f6ad311fa42c1a5918d,000000_2022-12-08_10-11-50_0_0,0,healthy
3,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,healthy,16000,10,05d3890a300e2841a4ff2e11829a08cc,000000_2022-12-08_10-11-50_0_0,0,healthy
4,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,faulty,16000,clean,e168eee455ed5e930c238b695a797210,000000_2022-12-08_10-11-50_0_0,1,faulty
5,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,faulty,16000,20,1d52461cb9c60b0cd69c0631e2fb59d9,000000_2022-12-08_10-11-50_0_0,1,faulty
6,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,faulty,16000,10,7902b76a6797c707edbabbb1a3f130ad,000000_2022-12-08_10-11-50_0_0,1,faulty
7,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,healthy,16000,clean,dabd80352308cd8847546d4e8c9ba25b,000000_2022-12-08_10-50-21_0_0,0,healthy
8,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,healthy,16000,30,1dac77664bc0a8b59a430a9ed44be361,000000_2022-12-08_10-50-21_0_0,0,healthy
9,data/cleaned/aero16k_noisy/000000_2022-12-08_1...,healthy,16000,20,2318aff0a4135f854375b2d4ecdbb7b5,000000_2022-12-08_10-50-21_0_0,0,healthy


In [20]:
test_loss, test_acc = model.evaluate(test_ds)
print("Overall Test Accuracy:", test_acc)


44/44 ━━━━━━━━━━━━━━━━━━━━ 15s 323ms/step - accuracy: 0.9699 - loss: 0.1183
Overall Test Accuracy: 0.9593437910079956


In [21]:
snr_levels = ['clean', 30, 20, 10]
results = []

for snr in snr_levels:
    subset = test_df[test_df['snr'] == snr]
    if subset.empty:
        continue
    subset.to_csv('data/manifests/temp.csv', index=False)
    snr_ds = make_dataset('data/manifests/temp.csv', shuffle=False)
    _, acc = model.evaluate(snr_ds, verbose=0)
    results.append((snr, acc))

results


[('clean', 0.9762532711029053)]

In [22]:
os.makedirs('results', exist_ok=True)
with open('results/metrics.txt', 'w') as f:
    f.write(f"Overall Test Accuracy: {test_acc}\n")
    for s, a in results:
        f.write(f"SNR {s}: {a}\n")


In [23]:
os.makedirs('models', exist_ok=True)
model.save('models/yamnet_fault_classifier.h5')


In [29]:
import os

paths = [
    "scripts/download_data.sh",
    "src/preprocess_all.py",
    "notebooks/AeroSound-FaultNet_YAMNet.ipynb",
    "results/metrics.txt",
    "data_splits/train.csv",
    "data_splits/val.csv",
    "data_splits/test.csv"
]

for p in paths:
    print(p, "→", os.path.exists(p))


scripts/download_data.sh → True
src/preprocess_all.py → True
notebooks/AeroSound-FaultNet_YAMNet.ipynb → True
results/metrics.txt → True
data_splits/train.csv → True
data_splits/val.csv → True
data_splits/test.csv → True


In [25]:
!find data -name "train.csv"
!find data -name "val.csv"
!find data -name "test.csv"


data/manifests/train.csv
data/manifests/val.csv
data/manifests/test.csv


In [27]:
import shutil

shutil.copy("data/manifests/train.csv", "data_splits/train.csv")
shutil.copy("data/manifests/val.csv", "data_splits/val.csv")
shutil.copy("data/manifests/test.csv", "data_splits/test.csv")

print("Copied successfully.")


Copied successfully.


In [28]:
paths = [
    "data_splits/train.csv",
    "data_splits/val.csv",
    "data_splits/test.csv"
]

for p in paths:
    print(p, "→", os.path.exists(p))


data_splits/train.csv → True
data_splits/val.csv → True
data_splits/test.csv → True


In [30]:
from google.colab import files
files.download("scripts/download_data.sh")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
from google.colab import files
files.download("src/preprocess_all.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>